# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открытие файла с данными и изучение общей информации

In [33]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
data.head(15) #просмотрим первые 15 строк таблицы


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [34]:
data.info() #оцениваем объем данных в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
Имена столбцов записаны корректно, однако имеется одинаковое количество незаполненных данных в столбцах 'days_employed' и 'total_income'.
Также общий просмотр таблицы выявил наличие отрицательных данных в столбце 'days_employed', что не может быть в реальной жизни.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [35]:
data[data['days_employed'].isna()] #просматриваем какие строки заполнены пустыми данными


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [36]:
data.groupby('income_type')['days_employed'].min() #выявяем минимальные значения по столбцу трудового стажа

income_type
безработный        337524.466835
в декрете           -3296.759962
госслужащий        -15193.032201
компаньон          -17615.563266
пенсионер          328728.720605
предприниматель      -520.848083
сотрудник          -18388.949901
студент              -578.751554
Name: days_employed, dtype: float64

In [37]:
data.groupby('income_type')['days_employed'].max() #выявяем максимальные значения по столбцу трудового стажа

income_type
безработный        395302.838654
в декрете           -3296.759962
госслужащий           -39.954170
компаньон             -30.195337
пенсионер          401755.400475
предприниматель      -520.848083
сотрудник             -24.141633
студент              -578.751554
Name: days_employed, dtype: float64

In [38]:
data.groupby('income_type')['total_income'].max() #вычисляем максимальный ежемесячный доход по группам лиц

income_type
безработный        2.027225e+05
в декрете          5.382913e+04
госслужащий        9.104515e+05
компаньон          2.265604e+06
пенсионер          7.351033e+05
предприниматель    4.991631e+05
сотрудник          1.726276e+06
студент            9.820163e+04
Name: total_income, dtype: float64

In [39]:
data.groupby('income_type')['total_income'].min() #вычисляем минимальный ежемесячный доход по группам лиц

income_type
безработный         59956.991984
в декрете           53829.130729
госслужащий         29200.077193
компаньон           28702.812889
пенсионер           20667.263793
предприниматель    499163.144947
сотрудник           21367.648356
студент             98201.625314
Name: total_income, dtype: float64

In [40]:
data.groupby('income_type')['total_income'].median() #считаем медиану ежемесячного дохода группируя по типу занятости


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [41]:
data['total_income']=data['total_income'].fillna(0) #заменяю пустые значения на 0

In [42]:

def mean_total_income_group(row): #задаем функцию, которая меняет 0 в столбце 'total_income' на медиану по группе
        
    if row['total_income']==0:
        if 'безработный' in row['income_type']:
            row['total_income']=131339.751676
            return row['total_income']
        if 'пенсионер' in row['income_type']:
            row['total_income']=118514.486412
            return row['total_income']
        if 'в декрете' in row['income_type']:
            row['total_income']=53829.130729
            return row['total_income']
        if 'госслужащий' in row['income_type']:
            row['total_income']=150447.935283
            return row['total_income']
        if 'компаньон' in row['income_type']: 
            row['total_income']=172357.950966
            return row['total_income']
        if 'предприниматель' in row['income_type']:
            row['total_income']=499163.144947
            return row['total_income']
        if 'сотрудник' in row['income_type']:
            row['total_income']=142594.396847
            return row['total_income']
        if 'студент' in row['income_type']:
            row['total_income']=98201.625314
            return row['total_income']

    else:
        return row['total_income']
        
   


In [43]:
data['total_income']=data.apply(mean_total_income_group, axis=1)
data[data['total_income'].isna()] #проверяем остались ли пустые строки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [44]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))#заменяем пустые данные в столбце на среднее арифмитическое
data[data['days_employed'].isna()] #проверяем остались ли пустые строки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
При просмотре строк с пустыми данными выявили, что столбцы с общим трудовым стажем и ежемесячным доходом пусты в одних и тех же строках.

В то же время в столбце с трудовым стажем выявлены отрицательные значения, которые логически не могут там быть. Скорее всего это связано с технической ошибкой. Однако получить информацию о причинах  потери данных невозможно.

Так как пропущенные данные, а также данные с аномальными отрицательными значениями с общим трудовым стажем не влияют на поставленный вопрос, принято решение заменить пустые данные на среднее арифмитическое.

Чтобы избежать аномальных выбросов в нижнюю или верхнюю строну, пропущенные данные в столбце с ежемесячным доходом были заменены на данные медианы по группам

### Замена типа данных

In [46]:
data['days_employed']=data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**
Столбец с общим трудовым стажем в днях был переведен в формат целых чисел, так как количество дней может быть только целым числом

### Обработка дубликатов

In [47]:
data['education'].value_counts() #проверили уникальные значения столбца с указанием образования

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [48]:
data['education']=data['education'].str.lower() #преобразовали символы в столбце 'education' в нижний регистр
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [49]:
data['family_status'].value_counts() #проверили уникальные значения столбца 'family_status'

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [51]:
data['income_type'].value_counts() #проверили уникальные значения столбца 'income_type'

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [52]:
data['purpose'].value_counts() #проверили уникальные значения столбца с указанием цели кредита

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [53]:
print(data.duplicated().sum()) #проверили наличие дубликатов, выявили их количество

71


In [54]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,-2326,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,покупка жилья для семьи
3290,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
4182,1,-2326,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
4851,0,365003,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба
5557,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,365003,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,дополнительное образование
21032,0,365003,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,заняться образованием
21132,0,-2326,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,ремонт жилью
21281,1,-2326,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,покупка коммерческой недвижимости


In [55]:
data=data.drop_duplicates().reset_index(drop=True) #удалили дубликаты

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**
При обработке в столбце с образованием были выявлены файлы с разным регистром, в связи с этим данный столбец был преобразован в нижний регистр.

Также в столбце с целями для кредита были выявлены повторяющиеся цели, которые будут также обработаны

В данных были выявлены строки-дубликаты, которые были удалены

### Лемматизация

In [57]:
from pymystem3 import Mystem
m = Mystem() 
def category_purpose(row): #создаем функцию, которая лемматизирует столбец с целями на кредит
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        row['purpose']='автомобиль'
        return row['purpose']
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ):
        row['purpose']='недвижимость'
        return row['purpose']
    if 'свадьба' in lem_purpose:
        row['purpose']='свадьба'
        return row['purpose']
    if 'образование' in lem_purpose:
        row['purpose']='образование'
        return row['purpose']
    else:
        return 'без категории'
 
data['purpose_category'] = data.apply(category_purpose, axis=1) #создаем столбец с переименованием категорий 
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

**Вывод**
Все цели кредита были разделены на 4 категории: недвижимость, автомобиль, образование и свадьба

### Категоризация данных

In [58]:
data['children'].value_counts() #проверили уникальные значени по количеству детей

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [59]:
data.loc[(data['children']==20), 'children'].count() #количество строк с аномальным значением 20


76

In [60]:
data.loc[(data['children']==-1), 'children'].count() #количество строк с аномальным значением -1

47

In [61]:
#меняем отрицательное значение -1 на 1
data.loc[(data['children']==-1), 'children'] = 1
data['children'].value_counts() #проверяем уникальные значения по количеству детей


0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [62]:
data.loc[(data['children']==20), 'children'] = 2 #меняем большое число 20 на 2
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [63]:
data['debt'].value_counts() #проверили уникальные значения столбца 'debt'


0    19713
1     1741
Name: debt, dtype: int64

In [64]:
def children_group(messages): #создаем функцию, которая классифицирует наличие детей на 3 группы
    if messages==0:
        return 'нет детей'
    if messages>=3:
        return 'многодетный'
    return '1-2 детей'
 
data['status_children']=data['children'].apply(children_group)
data_children_grouped = data.groupby('status_children').count()
print(data_children_grouped)


                 children  days_employed  dob_years  education  education_id  \
status_children                                                                
1-2 детей            6983           6983       6983       6983          6983   
многодетный           380            380        380        380           380   
нет детей           14091          14091      14091      14091         14091   

                 family_status  family_status_id  gender  income_type   debt  \
status_children                                                                
1-2 детей                 6983              6983    6983         6983   6983   
многодетный                380               380     380          380    380   
нет детей                14091             14091   14091        14091  14091   

                 total_income  purpose  purpose_category  
status_children                                           
1-2 детей                6983     6983              6983  
многодетный          

In [65]:
def total_income_group(messages): #создаем функцию, которая классифицирует доход исходя из среднедушевого дохода за 3 квартал 2021 года(по данным статистики)
    if messages<40000:
        return '1 низкий'
    if (messages>=40000) & (messages<100000):
        return '2 средний'
    if (messages>=100000) & (messages<150000):
        return '3 выше среднего'
    if (messages>=150000) & (messages<200000):
        return '4 высокий'
    return '5 очень высокий'
data['status_total_income']=data['total_income'].apply(total_income_group)
data_total_income_grouped=data.groupby('status_total_income').count()
print(data_total_income_grouped)

                     children  days_employed  dob_years  education  \
status_total_income                                                  
1 низкий                  125            125        125        125   
2 средний                4338           4338       4338       4338   
3 выше среднего          7160           7160       7160       7160   
4 высокий                4764           4764       4764       4764   
5 очень высокий          5067           5067       5067       5067   

                     education_id  family_status  family_status_id  gender  \
status_total_income                                                          
1 низкий                      125            125               125     125   
2 средний                    4338           4338              4338    4338   
3 выше среднего              7160           7160              7160    7160   
4 высокий                    4764           4764              4764    4764   
5 очень высокий              5067        

**Вывод** При категоризации данных по количеству детей были выявлены странные значения: (количество детей - отрицательное число, а также количество детей равно 20, что очень редко в нынешней ситуации).

Скорее всего произошла техническая ошибка, поэтому количество детей с отрицательным значением -1 было заменено на 1, а количество детей с 20 детьми заменено на 2.

Данные классифицировали следующим образом: нет детей, 1-2 ребенка, больше 3 (включительно), так как в России многодетными считаются семьи с 3 и больше детьми

Ежемесячный доход классифицировали исходя из среднедушевого денежного дохода, рассчитанного Росстатом, как минимального и дальнейшей арифмитической градацией

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [66]:
pivot_table_status_children_category=data.pivot_table(index='status_children', columns='debt', values='days_employed', aggfunc='count')
pivot_table_status_children_category['percent'] = pivot_table_status_children_category[1]/(pivot_table_status_children_category[1]+ pivot_table_status_children_category[0]) * 100
print(pivot_table_status_children_category)

debt                 0     1   percent
status_children                       
1-2 детей         6336   647  9.265359
многодетный        349    31  8.157895
нет детей        13028  1063  7.543822


**Вывод**
Количество клиентов, не имеющих детей и имеющих просрочки по кредитам, меньше, чем количество клиентов, имеющих детей.
В то же время клиенты, имеющих больше 3х детей, более ответственны к погашению кредита, чем клиенты с 1 или 2 детьми. Возможно они уже готовы ко всем необходимым или внезапным тратам, связанными с детьми, подготовились к этому заранее. Семьи с 1 или 2 ребенком еще не готовы полностью оптимизировать затраты.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [67]:
pivot_table_family_status_category=data.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')
pivot_table_family_status_category['percent'] = pivot_table_family_status_category[1]/(pivot_table_family_status_category[1]+ pivot_table_family_status_category[0]) * 100
print(pivot_table_family_status_category)

debt                       0    1   percent
family_status                              
Не женат / не замужем   2536  274  9.750890
в разводе               1110   85  7.112971
вдовец / вдова           896   63  6.569343
гражданский брак        3763  388  9.347145
женат / замужем        11408  931  7.545182


**Вывод**
Самыми исполнительными в погашении кредитов являются лица, которые познали супружество, либо находятся в браке на сегодня.

Можно рискнуть сделать вывод, что люди, которые не несут законной ответственности перед супругом или супругой, менее ответственны и перед банком

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [68]:
pivot_table_total_income_category=data.pivot_table(index='status_total_income', columns='debt', values='days_employed', aggfunc='count')
pivot_table_total_income_category['percent'] = pivot_table_total_income_category[1]/(pivot_table_total_income_category[1]+ pivot_table_total_income_category[0]) * 100
print(pivot_table_total_income_category)

debt                    0    1   percent
status_total_income                     
1 низкий              115   10  8.000000
2 средний            3994  344  7.929922
3 выше среднего      6536  624  8.715084
4 высокий            4359  405  8.501259
5 очень высокий      4709  358  7.065325


**Вывод**
Самыми дисциплинированными по выплате кредита являются лица, имеющие очень высокий доход.
Дисциплинированность остальных лиц исчисляется примерно на одном уровне

- Как разные цели кредита влияют на его возврат в срок?

In [69]:
pivot_table_purpose_category = data.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_purpose_category['percent'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[1]+ pivot_table_purpose_category[0]) * 100
print(pivot_table_purpose_category)

debt                  0    1   percent
purpose_category                      
автомобиль         3903  403  9.359034
недвижимость      10029  782  7.233373
образование        3643  370  9.220035
свадьба            2138  186  8.003442


**Вывод**

Существенной разницы между долей людей, имеющих задолженности по выплате кредита, в зависимости от целей этого кредита не наблюдается.

Однако люди, берущие кредит на недвижимость и свадьбу, более дисциплинированы, чем кредиторы, имеющие цели приобрести автомобиль или получить образование.



## Шаг 4. Общий вывод


Для исследования влияет ли семейное положение и количество детей клиента банков на факт погашения кредита в срок был проведен анализ  входных данных от банка — статистика о платёжеспособности клиентов.

1. Обработка данных.

Обработка данных показала наличие незаполненных данных, дубликатных строк, а также аномальных значений(отрицательных и не подходящих по смыслу)

Так как возможность прокомментировать данные значения отсутствовала, были приняты следующие решения:

- заменены отрицательные или отсутствующие данные о трудовом стаже на среднее арифмитическое. Данное решение принято так как цфры не участвовали в решении задачи;

- отсутсвующие данные о доходе были заменены на медианное, в соответсвии с их типом занятости

2. Вывод, рекомендации заказчику.

Были проверены следующие гипотезы, 2 из которых подтвердились полностью, а 2 - частично.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
  
Семьи без детей более дисциплинированы с возвратом кредита во время.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Самыми исполнительными в погашении кредитов являются лица, которые познали супружество (вдовцы, в разводе), либо находятся в браке на сегодня.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Самыми дисциплинированными по выплате кредита являются лица, имеющие очень высокий доход. Однако данных лиц не так много, по сравнению с другими. Дисциплинированность остальных лиц исчисляется примерно на одном уровне

- Как разные цели кредита влияют на его возврат в срок?


Люди, берущие кредит на недвижимость, более дисциплинированы, чем кредиторы, имеющие цели организовать свадьбу, приобрести автомобиль или получить образование.


**Итог** Наблюдается зависимость количества детей от выплаты кредита, а также семейного состояния и частично от цели кредита.В то же время высокой зависимости между уровнем дохода и выплатой кредита не наблюдается.

На основе предложенных данных самым дисциплинированным лицом по выплате кредита является вдовец(вдова), не имеющих детей и с очень высоким уровнем дохода. Такое лицо обычно берет кредит на недвижимость или свадьбу. 

Высоким доверием также пользуются семейные лица, имеющие много детей. Данной категории скорее всего пригодится кредит на недвижимость, который может быть также выдан.




